In [ ]:
from utils import tensorSum, xi, gaussianCubature
from numpy import sqrt

def eulerian_cubature_construction(dimension : int = 3, drift : bool = True):
    """Computes the degree-seven cubature formula for Brownian motion given in Theorem 5.8"""
    
    z = gaussianCubature(degree = 7, dimension = dimension)
    zz = gaussianCubature(degree = 3, dimension = dimension ** 2)
    zz = [([[None if i == 0 or j == 0 else x[0][1:][3 * (i - 1) + (j - 1)] for j in range(4)] for i in range(4)], x[1]) for x in zz]
    zzz = gaussianCubature(degree = 3, dimension = 1)

    cubature = []
    for z_n, z_weight in z:
        for z_m, zz_weight in zz:
            for z_r, zzz_weight in zzz:
                L = tensorSum([
                    z_n[i] * xi([i])
                    for i in range(1,dimension + 1)
                ])
                L += tensorSum([
                    ((1 / sqrt(3)) * z_n[i] * z_m[j][j] + (1 / sqrt(6)) * z_m[i][j]) * xi([i,j])
                    + (1 / 2) * z_n[i] * xi([i,j,j])
                    for i in range(1, dimension + 1) for j in range(1, dimension+1)
                ])
                L += tensorSum([
                    (1 / sqrt(6)) * z_m[i][j] * z_n[k] * z_r[1] * xi([i,j,k])
                    + (1 / (2 * sqrt(3))) * z_n[i] * z_m[j][j] * xi([i,j,k,k])
                    + (1 / (4 * sqrt(3))) * z_n[i] * z_m[k][k] * xi([i,j,j,k])
                    + (1 / 12) * z_n[i] * xi([i,j,j,k,k])
                    + (1 / 24) * z_n[j] * xi([i,i,j,k,k])
                    for i in range(1,dimension+1) for j in range(1,dimension+1) for k in range(1,dimension+1)
                ])
                if drift:
                    L += xi([0])
                    L += tensorSum([
                        sqrt(1/3) * z_m[i][i] * xi([0,i])
                        + (1/2) * xi([0,i,i])
                        for i in range(1,dimension+1)
                    ])
                    L += tensorSum([
                        (1/12) * xi([0,i,i,j,j])
                        + (1/24) * xi([i,i,0,j,j])
                        for i in range(1,dimension+1) for j in range(1,dimension+1)
                    ])
                
                cubature.append((L, z_weight * zz_weight * zzz_weight))
    return cubature

In [ ]:
# Build cubature formula in tensor space
cubature = eulerian_cubature_construction()

In [ ]:
# Compute the RHS of Equation (9)
from utils import pi, tensorSum
from free_lie_algebra import exp, Elt

degree = 7
dimension = 3

rhs = tensorSum([w * exp(L, maxLevel = degree) for L, w in cubature])
rhs = pi(rhs, 7)

In [ ]:
# Validate cubature formula by showing RHS is equal to expected signature
from free_lie_algebra import distance
from utils import expected_signature

distance(rhs, expected_signature())

1.0645737525107601e-14